In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import requests
import datetime
import re
import json
from datetime import date, timedelta
from datetime import datetime
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,array, expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,to_timestamp,from_utc_timestamp, regexp_replace,posexplode_outer
client = bigquery.Client(location="us-central1")
print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

spark = SparkSession.builder.appName("YT1-allocation-detailed").getOrCreate()

Client creating using default project: saas-analytics-io


In [2]:
environment = 'YT1'

In [4]:
if environment == 'YT1':
    project_number = '260718309650'

In [5]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_resource_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number)= '%s' and date(usage_start_time) between '2023-04-01' and '2023-04-30' "%(project_number))\
.load()

In [6]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("service.description").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type"),
                                          col("resource.name").alias('resource_name'),
                                          col("resource.global_name").alias('resource_global_name'),
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type"),
                                          col('resource_name'),
                                          col('resource_global_name'),
                                         ) 
GCP_cost_table.show()

+-------------------+---------------+-------------------+---------------+--------+------------+----------+--------------+----------------+--------------+--------------+----------------+--------------+---------+--------------------+--------------------+
|service_description|sku_description|   usage_start_time|     id_project|    cost|usage_amount|usage_unit|credits_1_name|credits_1_amount|credits_1_type|credits_2_name|credits_2_amount|credits_2_type|cost_type|       resource_name|resource_global_name|
+-------------------+---------------+-------------------+---------------+--------+------------+----------+--------------+----------------+--------------+--------------+----------------+--------------+---------+--------------------+--------------------+
|    Cloud Scheduler|           Jobs|2023-04-30 07:00:00|Cloud Scheduler|0.109677|        34.0|  requests|          null|            null|          null|          null|            null|          null|  regular|                null|          

In [53]:
GCP_cost_table = GCP_cost_table.withColumn("component",when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard%')), 'Mongo DB Platform - Shard')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm%')),'Mongo DB Platform - Shard')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-router%')),'Mongo DB Platform - Router')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-config%')),'Mongo DB Platform - config')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-analytics%')),'Mongo DB Analytics')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-temp%')),'Mongo DB Platform - Temp')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%kafka%')),'Kafka Browser')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%gke%')),'Containers (gke)')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%report%')),'Reports')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%nfs%')),'NFS')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%rabbit%')),'Rabbit')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%sftp%')), 'SFTP')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%vpn%')),'VPN')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%manager%')),'Manager')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%postgres%')),'Postgres')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%powerbi%')),'Power BI')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('ytem%')),'Microservices') 
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('monitoring%')),'Monitoring') 
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('analytics%')),'Microservices Analytics')
.when((GCP_cost_table['resource_name'].isNotNull())&(GCP_cost_table['resource_name'].like('%vpc--connector%')),'VPC Connector')
.when((GCP_cost_table['resource_global_name'].like('%forwarding_rule%')),'Forwarding Rule')
.when((GCP_cost_table['resource_global_name'].like('%cloudfunctions%')),'Cloud Functions')
.when((GCP_cost_table['resource_global_name'].like('%scxyt1cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%ytemyt1cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%cloudsql%')),'Cloud SQL')
.when((GCP_cost_table['resource_global_name'].like('%k8s-mojix-io%')),'Containers (gke)'))

In [54]:
GCP_cost_table.groupBy('component').sum().show()

+--------------------+--------------------+--------------------+---------------------+---------------------+
|           component|           sum(cost)|   sum(usage_amount)|sum(credits_1_amount)|sum(credits_2_amount)|
+--------------------+--------------------+--------------------+---------------------+---------------------+
|          Monitoring|    10.5340159999999|7.460488959001664E17|                 null|                 null|
|                null|          705.994229|2.034977643000223...|  -207.38100000000045|                 null|
|Mongo DB Platform...|  235.40457599999112|5.862337505072295...|           -38.118951|                 null|
|Mongo DB Platform...|  1341.2316889999568|1.082888103370324...|   -203.3016759999999|                 null|
|                 VPN|   41.87063500000033|7.145934088593469...|            -9.312679|                 null|
|Mongo DB Platform...|   18529.36310700483|3.646669572030175E20|  -1099.6263209999984|                 null|
|       Kafka Brows

In [55]:
GCP_cost_table_analysis = GCP_cost_table.filter((GCP_cost_table.component.isNull()))
GCP_cost_table_analysis.groupBy(['resource_name','resource_global_name']).sum().orderBy(col("sum(cost)").desc()).show(truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------+-----------------+---------------------+---------------------+---------------------+
|resource_name                   |resource_global_name                                                                          |sum(cost)        |sum(usage_amount)    |sum(credits_1_amount)|sum(credits_2_amount)|
+--------------------------------+----------------------------------------------------------------------------------------------+-----------------+---------------------+---------------------+---------------------+
|null                            |null                                                                                          |705.9941990000001|2.0349776421541958E19|-207.38100000000037  |null                 |
|acee86cd78e374e3cb6bf78a43a3633f|//compute.googleapis.com/projects/260718309650/regions/us-central1/address/5356733001962373019|3.0E-5         

In [56]:
GCP_cost_table = GCP_cost_table.withColumn("SubComponent",when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-1%')),'Mongo DB Platform - Shard 1')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-2%')),'Mongo DB Platform - Shard 2')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-3%')),'Mongo DB Platform - Shard 3')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-4%')),'Mongo DB Platform - Shard 4')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-5%')),'Mongo DB Platform - Shard 5')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-6%')),'Mongo DB Platform - Shard 6')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-shard-7%')),'Mongo DB Platform - Shard 7')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm002%')),'Mongo DB Platform - Shard 0')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm001%')),'Mongo DB Platform - Shard 0')
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('%mongo-vm003%')),'Mongo DB Platform - Shard 0')                                          
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('ytem%')),col('resource_name'))
.when((GCP_cost_table['component'].isNotNull())&(GCP_cost_table['resource_name'].like('analytics%')),col('resource_name'))
.otherwise('N/A'))

In [57]:
GCP_cost_table = GCP_cost_table.withColumn('usage_start_time_LOS_ANGELES', from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles"))


In [66]:
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["cost",'credits_1_amount','credits_2_amount'])
GCP_cost_table = GCP_cost_table.na.fill(value='N/A',subset=['SubComponent'])

In [67]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_discounts']+GCP_cost_table['credits_2_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("discounts_c2",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_1_amount']))
#GCP_cost_table = GCP_cost_table.withColumn("discounts_c2",when(GCP_cost_table['credits_2_type'].like('%DISCOUNT%'),GCP_cost_table['credits_1_amount']))


In [68]:
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [69]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when(GCP_cost_table['sku_description'].like('%SSD%'),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("compute_cost",when((~(GCP_cost_table['sku_description'].like('%SSD%'))),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts', GCP_cost_table['cost'] + GCP_cost_table['credits_discounts'])
GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts_promotion', GCP_cost_table['cost'] + GCP_cost_table['credits_discounts']+ GCP_cost_table['credits_promotion'])


#GCP_cost_table = GCP_cost_table.withColumn('total_after_discounts',GCP_cost_table['cost'] + GCP_cost_table['credits_1_amount']+ GCP_cost_table['credits_2_amount'])
#GCP_cost_table = GCP_cost_table.withColumn('amount_credits', GCP_cost_table['credits_1_amount']+ GCP_cost_table['credits_2_amount'])
   
    #GCP_cost_table.loc[(~(GCP_cost_table['sku_description'].str.contains('SSD'))),'processing_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    #GCP_cost_table.loc[(GCP_cost_table['sku_description'].str.contains('SSD')),'storage_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    #GCP_cost_table.loc[(~(GCP_cost_table['sku_description'].str.contains('SSD'))),'total_processing_after_discount']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1']+ GCP_cost_table['amount_credits_2']


In [ ]:
GCP_cost_table.groupBy('credits_1_type').sum('credits_1_amount').show()

In [74]:
GCP_cost_table.printSchema()

root
 |-- service_description: string (nullable = true)
 |-- sku_description: string (nullable = true)
 |-- usage_start_time: timestamp (nullable = true)
 |-- id_project: string (nullable = true)
 |-- cost: double (nullable = false)
 |-- usage_amount: double (nullable = true)
 |-- usage_unit: string (nullable = true)
 |-- credits_1_name: string (nullable = true)
 |-- credits_1_amount: double (nullable = false)
 |-- credits_1_type: string (nullable = true)
 |-- credits_2_name: string (nullable = true)
 |-- credits_2_amount: double (nullable = false)
 |-- credits_2_type: string (nullable = true)
 |-- cost_type: string (nullable = true)
 |-- resource_name: string (nullable = true)
 |-- resource_global_name: string (nullable = true)
 |-- component: string (nullable = true)
 |-- SubComponent: string (nullable = false)
 |-- usage_start_time_LOS_ANGELES: timestamp (nullable = true)
 |-- credits_discounts: double (nullable = false)
 |-- credits_promotion: double (nullable = false)
 |-- storage

In [ ]:
bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)

In [76]:
GCP_cost_table.write.format('bigquery') \
    .option('table', 'saas-analytics-io.processed.yt1_enh_cost_table_v2') \
    .mode('append') \
    .save()